# Quick Intro to Large Language Models

In [36]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

# Maximum number of tokens

In [5]:
from langchain.llms import OpenAI

# Before executing the following code, make sure to have
# your OpenAI key saved in the “OPENAI_API_KEY” environment variable.
# Initialize the LLM
llm = OpenAI(model_name="text-davinci-003")

# Define the input text
input_text = "Imagine a world in which humans have successfully established a colony on Mars. The Mars colony, named TerraNova, is a self-sustaining community that houses thousands of inhabitants. The colony relies on advanced technology, resource management, and scientific research to thrive in the harsh Martian environment. Describe the key infrastructure and systems that enable TerraNova to function effectively and support the needs of its residents. Discuss the transportation network, communication systems, power generation, food production, waste management, and the overall governance structure of the colony. Additionally, explore the challenges that the Mars colony faces in terms of sustainability, maintaining a habitable environment, and ensuring the well-being of its inhabitants. Provide insights into the innovations and solutions that have been implemented to overcome these challenges. Finally, speculate on the potential long-term impacts of a successful Mars colony on human civilization, space exploration, and the future of our species. Consider the scientific, social, and ethical implications of establishing a permanent settlement on another planet. Your response should be comprehensive, well-reasoned, and supported by both scientific knowledge and creative imagination."

# Determine the maximum number of tokens from documentation
max_tokens = 4097

# Split the input text into chunks based on the max tokens
text_chunks = split_text_into_chunks(input_text, max_tokens)

# Process each chunk separately
results = []
for chunk in text_chunks:
    result = llm.process(chunk)
    results.append(result)

# Combine the results as needed
final_result = combine_results(results)

NameError: name 'split_text_into_chunks' is not defined

# Token distribution and predicting the next token

In [12]:
from langchain.llms import OpenAI

text = 'What would be a good company name for a company that makes colorful socks?'

llm = OpenAI(model_name = 'text-davinci-003', temperature = 0.7)

print(llm(text))



Happy Sockery.


# Tracking token usage ->Using callbacks

In [13]:
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback

llm = OpenAI(model_name="text-davinci-003", n=2, best_of=2)

with get_openai_callback() as cb:
    result = llm("Tell me a joke")
    print(cb)

Tokens Used: 47
	Prompt Tokens: 4
	Completion Tokens: 43
Successful Requests: 1
Total Cost (USD): $0.00094


In [19]:
print(result)



Q: What did one ocean say to the other ocean?
A: Nothing, they just waved.


# Few-shot learning

In [20]:
from langchain import PromptTemplate
from langchain import FewShotPromptTemplate

# create our examples
examples = [
    {
        "query": "What's the weather like?",
        "answer": "It's raining cats and dogs, better bring an umbrella!"
    }, {
        "query": "How old are you?",
        "answer": "Age is just a number, but I'm timeless."
    }
]

# create an example template
example_template = """
User: {query}
AI: {answer}
"""

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)
# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are excerpts from conversations with an AI
assistant. The assistant is known for its humor and wit, providing
entertaining and amusing responses to users' questions. Here are some
examples:
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

# now create the few-shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

In [32]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain

# load the model
chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

chain = LLMChain(llm=chat, prompt=few_shot_prompt_template)

chain.run("What's the meaning of life?")

"That's a deep question. Let me consult my philosopher algorithm... Ah, it says the meaning of life is to enjoy the journey and make the most of every moment. And also to eat lots of pizza."

# Examples with Easy Prompts: Text Summarization, Text Translation, and Question Answering

In [34]:
from langchain import PromptTemplate

template = """Question: {question}

Answer: """

prompt = PromptTemplate(
        template=template,
    input_variables=['question']
)

# user question
question = "What is the capital city of France?"

In [37]:
from langchain import HuggingFaceHub, LLMChain

# initialize Hub LLM
hub_llm = HuggingFaceHub(
        repo_id='google/flan-t5-large',
    model_kwargs={'temperature':0}
)

# create prompt template > LLM chain
llm_chain = LLMChain(
    prompt=prompt,
    llm=hub_llm
)

# ask the user question about the capital of France
print(llm_chain.run(question))

paris


## Asking multiple questions

In [38]:
qa = [
    {'question': "What is the capital city of France?"},
    {'question': "What is the largest mammal on Earth?"},
    {'question': "Which gas is most abundant in Earth's atmosphere?"},
    {'question': "What color is a ripe banana?"}
]
res = llm_chain.generate(qa)
print( res )

generations=[[Generation(text='paris', generation_info=None)], [Generation(text='giraffe', generation_info=None)], [Generation(text='nitrogen', generation_info=None)], [Generation(text='yellow', generation_info=None)]] llm_output=None run=RunInfo(run_id=UUID('62633da2-fd9d-41db-8c91-28daf0697f04'))


In [45]:
multi_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""
long_prompt = PromptTemplate(template=multi_template, input_variables=["questions"])

llm2 = OpenAI(model_name = 'text-davinci-003', temperature=0.7)

llm_chain = LLMChain(
    prompt=long_prompt,
    llm=llm2
)

qs_str = (
    "What is the capital city of France?\n" +
    "What is the largest mammal on Earth?\n" +
    "Which gas is most abundant in Earth's atmosphere?\n" +
        "What color is a ripe banana?\n"
)
llm_chain.run(qs_str)

'Paris\nBlue Whale\nNitrogen\nYellow'

## Text summarizer

In [46]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [47]:
summarization_template = "Summarize the following text to one sentence: {text}"
summarization_prompt = PromptTemplate(input_variables=["text"], template=summarization_template)
summarization_chain = LLMChain(llm=llm, prompt=summarization_prompt)

In [48]:
text = "LangChain provides many modules that can be used to build language model applications. Modules can be combined to create more complex applications, or be used individually for simple applications. The most basic building block of LangChain is calling an LLM on some input. Let’s walk through a simple example of how to do this. For this purpose, let’s pretend we are building a service that generates a company name based on what the company makes."
summarized_text = summarization_chain.predict(text=text)

In [49]:
print(summarized_text)

LangChain offers various modules for building language model applications, which can be used individually or combined to create complex applications, with the most basic building block being calling an LLM on input, as demonstrated in a simple example of building a company name generator.


## Text translation

In [50]:
translation_template = "Translate the following text from {source_language} to {target_language}: {text}"
translation_prompt = PromptTemplate(input_variables=["source_language", "target_language", "text"], template=translation_template)
translation_chain = LLMChain(llm=llm, prompt=translation_prompt)

In [53]:
source_language = "English"
target_language = "Kiswahili"
text = "Your text here"
translated_text = translation_chain.predict(source_language=source_language, target_language=target_language, text=text)

In [54]:
print(translated_text)

Maandishi yako hapa
